In [47]:
import pandas as pd
import torch
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification, AutoModelForTokenClassification, AutoTokenizer, RagTokenizer, RagRetriever, RagSequenceForGeneration
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import re

In [62]:
df = pd.read_csv('FIR_DATASET(updated).csv')

In [63]:
df.drop("URL",axis=1)

,Description,Offense,Punishment,Cognizable,Bailable,Court,section
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,140
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session,127
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,128
3,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class,129
4,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,130
...,...,...,...,...,...,...,...
440,Description of IPC Section 508\nAccording to s...,Act caused by inducing a person to believe tha...,1 Year or Fine or Both,Non-Cognizable,Bailable,Any Magistrate,508
441,Description of IPC Section 509\nAccording to s...,Uttering any word or making any gesture intend...,Simple imprisonment for 3 years + Fine,Cognizable,Bailable,Any Magistrate,509
442,Description of IPC Section 510\nAccording to s...,"Appearing in a public Place, etc., in a State ...",Simple Imprisonment for 24 hours or Fine or Both,Non-Cognizable,Bailable,Any Magistrate,510
443,Description of IPC Section 511\nAccording to s...,Attempting to commit offences punishable with ...,Imprisonment for Life or Imprisonment not exce...,Same As for Offence attempted,Same As for Offence attempted,Same As for Offence attempted,511


In [64]:
df.head()

,URL,Description,Offense,Punishment,Cognizable,Bailable,Court,section
0,https://lawrato.com/indian-kanoon/ipc/section-140,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,Cognizable,Bailable,Any Magistrate,140
1,https://lawrato.com/indian-kanoon/ipc/section-127,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,Cognizable,Non-Bailable,Court of Session,127
2,https://lawrato.com/indian-kanoon/ipc/section-128,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,128
3,https://lawrato.com/indian-kanoon/ipc/section-129,Description of IPC Section 129\nAccording to s...,Public servant negligently suffering prisoner ...,Simple Imprisonment 3 Years + Fine,Cognizable,Bailable,Magistrate First Class,129
4,https://lawrato.com/indian-kanoon/ipc/section-130,Description of IPC Section 130\nAccording to s...,"Aiding escape of, rescuing or harbouring, such...",Imprisonment for Life or 10 Years + Fine,Cognizable,Non-Bailable,Court of Session,130


In [65]:
section_data = df['Description']
offense_data = df['Description']
offense_labels = df['Offense']


In [66]:
train_data, test_data, train_labels, test_labels = train_test_split(
    section_data, df['section'], test_size=0.2, random_state=42
)

In [67]:
bert_model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = TFBertForSequenceClassification.from_pretrained(bert_model_name)


Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
train_encodings = bert_tokenizer(list(train_data), truncation=True, padding=True, return_tensors='tf')
test_encodings = bert_tokenizer(list(test_data), truncation=True, padding=True, return_tensors='tf')


In [ ]:
optimizer_bert = tf.keras.optimizers.Adam(learning_rate=5e-5)
bert_model.compile(optimizer=optimizer_bert, loss=bert_model.compute_loss, metrics=['accuracy'])
labels_bert = tf.keras.utils.to_categorical(train_labels, num_classes=len(df['section'].unique()))
history_bert = bert_model.fit(train_encodings, labels_bert, epochs=3, batch_size=8)


In [ ]:
bert_model.evaluate(test_encodings, tf.keras.utils.to_categorical(test_labels, num_classes=len(df['section'].unique())))


In [ ]:
llama2_model_path = 'path_to_llama2_model'  # Replace with the actual path to your LLama-2 model files
tokenizer_llama2 = AutoTokenizer.from_pretrained(llama2_model_path)
model_llama2 = AutoModelForTokenClassification.from_pretrained(llama2_model_path)

llama2_encodings = tokenizer_llama2(list(offense_data), truncation=True, padding=True, return_tensors='pt')


In [ ]:
optimizer_llama2 = torch.optim.AdamW(model_llama2.parameters(), lr=5e-5)
labels_llama2 = torch.tensor(list(offense_labels))  # Assuming you have labels for LLama-2
optimizer_llama2.zero_grad()
outputs_llama2 = model_llama2(**llama2_encodings, labels=labels_llama2)
loss_llama2 = outputs_llama2.loss
loss_llama2.backward()
optimizer_llama2.step()

In [ ]:
model_llama2.eval()
with torch.no_grad():
    logits_llama2 = model_llama2(**llama2_encodings).logits


In [ ]:
predicted_labels_bert = bert_model.predict(test_encodings)
predicted_labels_bert = tf.argmax(predicted_labels_bert['logits'], axis=1)


In [ ]:
predicted_labels_llama2 = torch.argmax(logits_llama2, dim=1)


In [ ]:
rag_tokenizer = RagTokenizer.from_pretrained("facebook/rag-token-base")
rag_retriever = RagRetriever.from_pretrained("facebook/rag-token-base")
rag_model = RagSequenceForGeneration.from_pretrained("facebook/rag-token-base")


In [ ]:
user_input = input("Enter your statement: ")


In [ ]:
predicted_section = bert_model.predict(bert_tokenizer([user_input], truncation=True, padding=True, return_tensors='tf'))['logits']
predicted_section = df['section'].unique()[tf.argmax(predicted_section, axis=1).numpy()[0]]


In [ ]:
context = f"Statement: '{user_input}' Predicted Section: {predicted_section} Offense: {predicted_offense}"
input_ids = rag_tokenizer(context, return_tensors="pt").input_ids


In [ ]:
generated_output = rag_model.generate(input_ids)
generated_text = rag_tokenizer.decode(generated_output[0], skip_special_tokens=True)


In [ ]:
additional_info = df[df['section'] == predicted_section].iloc[0]

In [ ]:
print("User Input:", user_input)
print("Predicted Section:", predicted_section)
print("Predicted Offense:", predicted_offense)
print("Generated Explanation:", generated_text)
print("Punishment:", additional_info['punishment'])
print("Court:", additional_info['court'])
print("Cognizable:", additional_info['cognizable'])
print("Bailable:", additional_info['bailable'])